In [22]:
# Import Libraries
import ee
import pandas as pd


In [21]:
# Initialise Earth engine
ee.Initialize()

In [25]:
def get_bands_per_pixel(start_date, end_date, polygon_coordinates, scale=10, batch_size=5000, cloud_percentage=20):

   
    roi = ee.Geometry.Polygon(polygon_coordinates)
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
                .filterDate(start_date, end_date) \
                .filterBounds(roi) \
                .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage))

    def select_and_scale_bands(image):
        bands = image.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']) \
                     .multiply(0.0001)  
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        cloud_cover = image.get('CLOUDY_PIXEL_PERCENTAGE')
        return bands.set({'datetime': date, 'cloud_percentage': cloud_cover})

    bands_collection = sentinel2.map(select_and_scale_bands)

    rows = []
    images_list = bands_collection.toList(bands_collection.size())
    num_images = bands_collection.size().getInfo()

    for i in range(num_images):
        image = ee.Image(images_list.get(i))
        date = image.get('datetime').getInfo()
        cloud_cover = image.get('cloud_percentage').getInfo()
        sampled = image.sample(
            region=roi,
            scale=scale,
            numPixels=batch_size,
            geometries=True
        ).getInfo()
        

        for feature in sampled['features']:
            coords = feature['geometry']['coordinates']
            properties = feature['properties']
            row = {
                'datetime': date,
                'longitude': coords[0],
                'latitude': coords[1],
                'cloud_percentage': cloud_cover
            }
            for band in ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']:
                row[band] = properties.get(band)
            rows.append(row)

    df = pd.DataFrame(rows)
    return df





In [27]:
start_date = '2023-01-01'
end_date = '2023-01-25'
cloud_percentage = 10  
polygon_coordinates = [
    [-74.1, 40.7],
    [-74.0, 40.7],
    [-74.0, 40.8],
    [-74.1, 40.8],
    [-74.1, 40.7] 
]


In [28]:
bands_df = get_bands_per_pixel(start_date, end_date, polygon_coordinates, cloud_percentage=cloud_percentage)


In [30]:
bands_df


,datetime,longitude,latitude,cloud_percentage,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B10,B11,B12
0,2023-01-15,-74.076544,40.738926,0.50081,0.1687,0.1148,0.0849,0.0798,0.1263,0.1296,0.1317,0.0743,0.1304,0.0420,0.0013,0.1192,0.1053
1,2023-01-15,-74.003597,40.703102,0.50081,0.1371,0.1077,0.0807,0.0520,0.0407,0.0281,0.0247,0.0213,0.0196,0.0080,0.0012,0.0080,0.0040
2,2023-01-15,-74.037936,40.720290,0.50081,0.1602,0.1143,0.0801,0.0582,0.1002,0.1001,0.1202,0.0628,0.1054,0.0348,0.0012,0.1266,0.0885
3,2023-01-15,-74.097356,40.709603,0.50081,0.1709,0.1573,0.1312,0.1308,0.1191,0.1378,0.1489,0.1360,0.1573,0.0477,0.0013,0.1393,0.1078
4,2023-01-15,-74.061709,40.791661,0.50081,0.1658,0.1237,0.1093,0.1084,0.1023,0.1273,0.1407,0.1290,0.1479,0.0476,0.0012,0.1332,0.0943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-01-15,-74.008187,40.743774,0.50081,0.1723,0.1012,0.0617,0.0456,0.0413,0.0405,0.0319,0.0285,0.0294,0.0340,0.0013,0.0200,0.0153
4996,2023-01-15,-74.048143,40.706290,0.50081,0.1403,0.1105,0.0820,0.0847,0.0853,0.0989,0.1150,0.1193,0.1356,0.0469,0.0013,0.1878,0.1085
4997,2023-01-15,-74.028316,40.758240,0.50081,0.1693,0.1250,0.0917,0.0884,0.1153,0.1178,0.1166,0.0760,0.1243,0.0495,0.0013,0.1213,0.0991
4998,2023-01-15,-74.013058,40.765174,0.50081,0.1427,0.1162,0.0938,0.0778,0.0654,0.0391,0.0338,0.0290,0.0236,0.0082,0.0009,0.0057,0.0041
